In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
from data import data_manager
from data.build_data import build_reid_dataset, build_par_dataset
from data.sampler import IdBasedSampler
from data.build_transform import build_transform
from torch.utils import data
from config.config_manager import _C as cfg
from data.build_loader import build_par_loader
from tools.logger import setup_logger
logger = setup_logger(".")

ImportError: cannot import name 'dist_idx_to_pair_idx'

### set config

In [2]:
cfg.merge_from_file("../par.yml")
cfg.DATASET.TRAIN_PATH = "/media/allen/mass/PAR"
cfg.DATASET.NAME = "par"

### load data by build_dataset

In [3]:
dataset = get_img_data(cfg)
train_trans = build_transform(cfg)
train_dataset = build_par_dataset(dataset.train, train_trans)
# sampler = IdBasedSampler(train_dataset, batch_size=cfg.INPUT.SIZE_TRAIN, num_instances=cfg.REID.SIZE_PERSON)
# loader = data.DataLoader(
#         train_dataset, batch_size=cfg.INPUT.SIZE_TRAIN, num_workers=1, pin_memory=True, sampler=sampler
#     )

2019-08-26 15:35:10,098 logger INFO: => PAR loaded
2019-08-26 15:35:10,099 logger INFO: Dataset statistics:
2019-08-26 15:35:10,100 logger INFO:   ------------------------------
2019-08-26 15:35:10,101 logger INFO:   subset   | # class | # images
2019-08-26 15:35:10,102 logger INFO:   ------------------------------
2019-08-26 15:35:10,102 logger INFO:   train    |      13 |   185887
2019-08-26 15:35:10,103 logger INFO:   val      |      13 |    94335
2019-08-26 15:35:10,103 logger INFO:   ------------------------------


In [4]:
train_dataset[0]

(tensor([[[ 0.2796,  0.2796,  0.2624,  ...,  1.3070,  1.3070, -2.1179],
          [ 0.2796,  0.2796,  0.2624,  ...,  1.3242,  1.3242, -2.1179],
          [ 0.2796,  0.2796,  0.2624,  ...,  1.3242,  1.3242, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[ 0.3627,  0.3627,  0.3452,  ...,  1.4657,  1.4657, -2.0357],
          [ 0.3627,  0.3627,  0.3452,  ...,  1.4657,  1.4657, -2.0357],
          [ 0.3803,  0.3803,  0.3627,  ...,  1.4832,  1.4832, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[ 0.6705,  0.6705,  0.6531,  ...,  1.6988,  1.6988, -1.8044],
          [ 0.6705,  0.6705,

In [5]:
from PIL import Image
import glob
import os
from tqdm import tqdm

In [6]:
root = "/media/allen/mass/PAR"
branchs = [i for i in os.listdir(root) if os.path.isdir(os.path.join(root,i))]

In [7]:
for branch in branchs:
    dst = os.path.join(root, branch, "cropped")
    if not os.path.exists(dst):
        os.mkdir(dst)

In [9]:
with open("/media/allen/mass/PAR/par_train_dataset.txt", 'w') as f:    
    for idx in tqdm(range(len(dataset.train))):
        path, bbox, attrs = dataset.train[idx]
        branch = path.split("/")[5]
        
        if 'wider' in branch:
            x1, y1, w, h = bbox
            bbox = (x1, y1, x1+w, y1+h)
        
        dst = os.path.join(root, branch, "cropped")

        filename = path.split(branch+"/")[-1].replace("/","_")
        imagename, extenstion = filename.split(".")
        img = Image.open(path)
        try:
            if bbox[0] != -1:
                img = img.crop(bbox)

            path = "{}/{}.{}".format(dst, imagename, extenstion)
            img.save(path)    
            line = "{} ".format(path.split("PAR/")[-1])
            for attr in attrs:
                line += "{} ".format(attr)
            f.write(line + "\n")
        except:
            print(idx)
            break

100%|██████████| 185887/185887 [24:02<00:00, 128.84it/s] 


In [20]:
with open("/media/allen/mass/PAR/par_test_dataset.txt", 'w') as f:    
    for idx in tqdm(range(len(dataset.val))):
        path, bbox, attrs = dataset.val[idx]
        branch = path.split("/")[5]
        
        if 'wider' in branch:
            x1, y1, w, h = bbox
            bbox = (x1, y1, x1+w, y1+h)
            
        dst = os.path.join(root, branch, "cropped")

        filename = path.split(branch+"/")[-1].replace("/","_")
        try:
            imagename, extenstion = filename.split(".")
        except:
            imagename, _, extenstion = filename.split(".")
            
        img = Image.open(path)
        if bbox[0] != -1:
            img = img.crop(bbox)

        path = "{}/{}.{}".format(dst, imagename, extenstion)
        img.save(path)    
        line = "{} ".format(path.split("PAR/")[-1])
        for attr in attrs:
            line += "{} ".format(attr)
        f.write(line + "\n")



  0%|          | 0/94335 [00:00<?, ?it/s]

  0%|          | 3/94335 [00:00<53:37, 29.32it/s]

  0%|          | 9/94335 [00:00<46:13, 34.00it/s]

  0%|          | 14/94335 [00:00<42:58, 36.58it/s]

  0%|          | 19/94335 [00:00<41:31, 37.86it/s]

  0%|          | 24/94335 [00:00<40:34, 38.73it/s]

  0%|          | 28/94335 [00:00<40:42, 38.61it/s]

  0%|          | 32/94335 [00:00<42:22, 37.09it/s]

  0%|          | 37/94335 [00:00<40:07, 39.16it/s]

  0%|          | 42/94335 [00:01<39:45, 39.53it/s]

  0%|          | 47/94335 [00:01<37:32, 41.86it/s]

  0%|          | 52/94335 [00:01<38:02, 41.32it/s]

  0%|          | 58/94335 [00:01<35:16, 44.54it/s]

  0%|          | 63/94335 [00:01<36:55, 42.55it/s]

  0%|          | 68/94335 [00:01<37:26, 41.96it/s]

  0%|          | 73/94335 [00:01<38:35, 40.71it/s]

  0%|          | 78/94335 [00:01<38:58, 40.31it/s]

  0%|          | 83/94335 [00:02<41:51, 37.54it/s]

  0%|          | 88/94335 [00:02<39:58, 39.30it/s]

  0%|          | 93/9

  2%|▏         | 1453/94335 [00:33<43:31, 35.57it/s]

  2%|▏         | 1459/94335 [00:33<41:06, 37.65it/s]

  2%|▏         | 1463/94335 [00:33<42:22, 36.53it/s]

  2%|▏         | 1468/94335 [00:33<41:04, 37.68it/s]

  2%|▏         | 1472/94335 [00:33<47:05, 32.86it/s]

  2%|▏         | 1476/94335 [00:34<48:11, 32.12it/s]

  2%|▏         | 1480/94335 [00:34<52:27, 29.50it/s]

  2%|▏         | 1484/94335 [00:34<50:35, 30.59it/s]

  2%|▏         | 1488/94335 [00:34<50:29, 30.65it/s]

  2%|▏         | 1492/94335 [00:34<51:27, 30.08it/s]

  2%|▏         | 1496/94335 [00:34<50:54, 30.39it/s]

  2%|▏         | 1500/94335 [00:34<47:45, 32.40it/s]

  2%|▏         | 1504/94335 [00:35<45:15, 34.18it/s]

  2%|▏         | 1508/94335 [00:35<50:54, 30.39it/s]

  2%|▏         | 1512/94335 [00:35<47:46, 32.39it/s]

  2%|▏         | 1516/94335 [00:35<47:21, 32.66it/s]

  2%|▏         | 1520/94335 [00:35<49:59, 30.94it/s]

  2%|▏         | 1524/94335 [00:35<1:06:49, 23.15it/s]

  2%|▏         | 1529/9433

  3%|▎         | 2914/94335 [01:06<28:47, 52.91it/s]

  3%|▎         | 2920/94335 [01:06<28:55, 52.68it/s]

  3%|▎         | 2926/94335 [01:06<29:01, 52.48it/s]

  3%|▎         | 2933/94335 [01:07<27:34, 55.25it/s]

  3%|▎         | 2940/94335 [01:07<26:53, 56.64it/s]

  3%|▎         | 2947/94335 [01:07<26:18, 57.89it/s]

  3%|▎         | 2953/94335 [01:07<26:17, 57.94it/s]

  3%|▎         | 2959/94335 [01:07<26:22, 57.73it/s]

  3%|▎         | 2965/94335 [01:07<28:19, 53.77it/s]

  3%|▎         | 2972/94335 [01:07<27:12, 55.95it/s]

  3%|▎         | 2979/94335 [01:07<26:09, 58.22it/s]

  3%|▎         | 2986/94335 [01:07<25:33, 59.57it/s]

  3%|▎         | 2993/94335 [01:08<25:24, 59.90it/s]

  3%|▎         | 3000/94335 [01:08<25:51, 58.88it/s]

  3%|▎         | 3006/94335 [01:08<26:53, 56.59it/s]

  3%|▎         | 3012/94335 [01:08<30:16, 50.28it/s]

  3%|▎         | 3018/94335 [01:08<33:29, 45.44it/s]

  3%|▎         | 3023/94335 [01:08<33:44, 45.11it/s]

  3%|▎         | 3028/94335 

  5%|▍         | 4544/94335 [01:38<27:05, 55.25it/s]

  5%|▍         | 4550/94335 [01:38<27:36, 54.19it/s]

  5%|▍         | 4556/94335 [01:38<28:22, 52.72it/s]

  5%|▍         | 4562/94335 [01:38<29:01, 51.54it/s]

  5%|▍         | 4568/94335 [01:38<29:20, 51.00it/s]

  5%|▍         | 4574/94335 [01:38<28:35, 52.31it/s]

  5%|▍         | 4580/94335 [01:38<27:41, 54.01it/s]

  5%|▍         | 4586/94335 [01:39<27:33, 54.28it/s]

  5%|▍         | 4592/94335 [01:39<27:44, 53.91it/s]

  5%|▍         | 4598/94335 [01:39<27:57, 53.48it/s]

  5%|▍         | 4604/94335 [01:39<27:36, 54.17it/s]

  5%|▍         | 4610/94335 [01:39<27:37, 54.14it/s]

  5%|▍         | 4616/94335 [01:39<27:36, 54.18it/s]

  5%|▍         | 4622/94335 [01:39<27:45, 53.88it/s]

  5%|▍         | 4628/94335 [01:39<28:14, 52.93it/s]

  5%|▍         | 4634/94335 [01:39<27:44, 53.88it/s]

  5%|▍         | 4640/94335 [01:40<27:51, 53.66it/s]

  5%|▍         | 4646/94335 [01:40<27:10, 55.01it/s]

  5%|▍         | 4652/94335 

  7%|▋         | 6149/94335 [02:09<28:21, 51.82it/s]

  7%|▋         | 6155/94335 [02:09<33:42, 43.60it/s]

  7%|▋         | 6160/94335 [02:09<35:28, 41.43it/s]

  7%|▋         | 6166/94335 [02:09<32:45, 44.85it/s]

  7%|▋         | 6172/94335 [02:09<31:02, 47.34it/s]

  7%|▋         | 6177/94335 [02:09<33:06, 44.37it/s]

  7%|▋         | 6182/94335 [02:10<34:41, 42.36it/s]

  7%|▋         | 6187/94335 [02:10<33:23, 44.00it/s]

  7%|▋         | 6193/94335 [02:10<31:15, 47.00it/s]

  7%|▋         | 6199/94335 [02:10<29:16, 50.19it/s]

  7%|▋         | 6205/94335 [02:10<27:57, 52.55it/s]

  7%|▋         | 6212/94335 [02:10<26:41, 55.01it/s]

  7%|▋         | 6218/94335 [02:10<27:38, 53.13it/s]

  7%|▋         | 6224/94335 [02:10<28:45, 51.05it/s]

  7%|▋         | 6230/94335 [02:11<30:06, 48.77it/s]

  7%|▋         | 6236/94335 [02:11<28:45, 51.07it/s]

  7%|▋         | 6242/94335 [02:11<27:50, 52.72it/s]

  7%|▋         | 6248/94335 [02:11<26:55, 54.51it/s]

  7%|▋         | 6254/94335 

...

 91%|█████████ | 85735/94335 [09:52<02:09, 66.26it/s]

 91%|█████████ | 85743/94335 [09:52<02:06, 67.98it/s]

 91%|█████████ | 85753/94335 [09:52<01:56, 73.65it/s]

 91%|█████████ | 85761/94335 [09:53<02:14, 63.80it/s]

 91%|█████████ | 85769/94335 [09:53<02:08, 66.46it/s]

 91%|█████████ | 85777/94335 [09:53<02:03, 69.37it/s]

 91%|█████████ | 85786/94335 [09:53<01:57, 73.05it/s]

 91%|█████████ | 85794/94335 [09:53<02:39, 53.69it/s]

 91%|█████████ | 85801/94335 [09:53<02:37, 54.05it/s]

 91%|█████████ | 85808/94335 [09:54<03:14, 43.82it/s]

 91%|█████████ | 85817/94335 [09:54<02:46, 51.07it/s]

 91%|█████████ | 85824/94335 [09:54<02:58, 47.55it/s]

 91%|█████████ | 85830/94335 [09:54<02:59, 47.31it/s]

 91%|█████████ | 85836/94335 [09:54<03:02, 46.50it/s]

 91%|█████████ | 85843/94335 [09:54<02:49, 50.21it/s]

 91%|█████████ | 85851/94335 [09:54<02:33, 55.16it/s]

 91%|█████████ | 85857/94335 [09:54<02:35, 54.63it/s]

 91%|█████████ | 85863/94335 [09:55<02:57, 47.75it/s]

 91%|█████

 93%|█████████▎| 87619/94335 [10:27<02:11, 50.93it/s]

 93%|█████████▎| 87625/94335 [10:27<02:21, 47.38it/s]

 93%|█████████▎| 87635/94335 [10:27<02:01, 55.30it/s]

 93%|█████████▎| 87644/94335 [10:27<02:01, 55.10it/s]

 93%|█████████▎| 87652/94335 [10:27<01:51, 60.02it/s]

 93%|█████████▎| 87659/94335 [10:27<01:55, 57.62it/s]

 93%|█████████▎| 87666/94335 [10:28<02:46, 40.09it/s]

 93%|█████████▎| 87671/94335 [10:28<03:03, 36.35it/s]

 93%|█████████▎| 87676/94335 [10:28<02:57, 37.56it/s]

 93%|█████████▎| 87681/94335 [10:28<03:00, 36.84it/s]

 93%|█████████▎| 87686/94335 [10:28<03:54, 28.31it/s]

 93%|█████████▎| 87694/94335 [10:29<03:10, 34.85it/s]

 93%|█████████▎| 87699/94335 [10:29<03:17, 33.57it/s]

 93%|█████████▎| 87704/94335 [10:29<04:02, 27.39it/s]

 93%|█████████▎| 87708/94335 [10:29<03:50, 28.76it/s]

 93%|█████████▎| 87712/94335 [10:29<03:45, 29.41it/s]

 93%|█████████▎| 87717/94335 [10:29<03:18, 33.32it/s]

 93%|█████████▎| 87724/94335 [10:29<02:47, 39.44it/s]

 93%|█████

 94%|█████████▍| 89094/94335 [11:04<04:31, 19.33it/s]

 94%|█████████▍| 89097/94335 [11:04<04:05, 21.36it/s]

 94%|█████████▍| 89101/94335 [11:04<03:31, 24.73it/s]

 94%|█████████▍| 89104/94335 [11:04<03:23, 25.70it/s]

 94%|█████████▍| 89107/94335 [11:04<03:31, 24.67it/s]

 94%|█████████▍| 89110/94335 [11:04<03:24, 25.50it/s]

 94%|█████████▍| 89113/94335 [11:04<03:26, 25.34it/s]

 94%|█████████▍| 89120/94335 [11:04<02:54, 29.88it/s]

 94%|█████████▍| 89124/94335 [11:05<03:15, 26.70it/s]

 94%|█████████▍| 89128/94335 [11:05<02:56, 29.45it/s]

 94%|█████████▍| 89133/94335 [11:05<02:34, 33.58it/s]

 94%|█████████▍| 89137/94335 [11:05<03:19, 26.06it/s]

 94%|█████████▍| 89143/94335 [11:05<02:45, 31.36it/s]

 95%|█████████▍| 89150/94335 [11:05<02:21, 36.62it/s]

 95%|█████████▍| 89155/94335 [11:05<02:20, 36.79it/s]

 95%|█████████▍| 89161/94335 [11:06<02:10, 39.65it/s]

 95%|█████████▍| 89166/94335 [11:06<02:23, 35.90it/s]

 95%|█████████▍| 89171/94335 [11:06<02:40, 32.16it/s]

 95%|█████

 96%|█████████▌| 90683/94335 [11:39<01:25, 42.63it/s]

 96%|█████████▌| 90688/94335 [11:39<01:38, 37.16it/s]

 96%|█████████▌| 90693/94335 [11:39<01:31, 39.94it/s]

 96%|█████████▌| 90699/94335 [11:40<01:23, 43.76it/s]

 96%|█████████▌| 90705/94335 [11:40<01:17, 46.71it/s]

 96%|█████████▌| 90712/94335 [11:40<01:10, 51.51it/s]

 96%|█████████▌| 90718/94335 [11:40<01:15, 47.70it/s]

 96%|█████████▌| 90725/94335 [11:40<01:09, 52.15it/s]

 96%|█████████▌| 90731/94335 [11:40<01:24, 42.46it/s]

 96%|█████████▌| 90736/94335 [11:40<01:22, 43.73it/s]

 96%|█████████▌| 90741/94335 [11:40<01:24, 42.46it/s]

 96%|█████████▌| 90747/94335 [11:41<01:20, 44.45it/s]

 96%|█████████▌| 90752/94335 [11:41<01:36, 36.99it/s]

 96%|█████████▌| 90759/94335 [11:41<01:24, 42.13it/s]

 96%|█████████▌| 90766/94335 [11:41<01:15, 46.99it/s]

 96%|█████████▌| 90772/94335 [11:41<01:50, 32.12it/s]

 96%|█████████▌| 90777/94335 [11:41<01:41, 35.02it/s]

 96%|█████████▌| 90784/94335 [11:42<01:28, 40.25it/s]

 96%|█████

 98%|█████████▊| 92784/94335 [12:13<00:28, 54.73it/s]

 98%|█████████▊| 92790/94335 [12:13<00:28, 54.34it/s]

 98%|█████████▊| 92799/94335 [12:13<00:25, 61.09it/s]

 98%|█████████▊| 92807/94335 [12:13<00:23, 64.29it/s]

 98%|█████████▊| 92816/94335 [12:13<00:22, 68.58it/s]

 98%|█████████▊| 92825/94335 [12:13<00:20, 72.05it/s]

 98%|█████████▊| 92833/94335 [12:14<00:20, 74.06it/s]

 98%|█████████▊| 92841/94335 [12:14<00:22, 67.82it/s]

 98%|█████████▊| 92849/94335 [12:14<00:23, 63.30it/s]

 98%|█████████▊| 92856/94335 [12:14<00:24, 60.03it/s]

 98%|█████████▊| 92863/94335 [12:14<00:24, 59.12it/s]

 98%|█████████▊| 92871/94335 [12:14<00:24, 60.44it/s]

 98%|█████████▊| 92880/94335 [12:14<00:22, 65.71it/s]

 98%|█████████▊| 92888/94335 [12:14<00:21, 67.96it/s]

 98%|█████████▊| 92895/94335 [12:15<00:24, 58.01it/s]

 98%|█████████▊| 92903/94335 [12:15<00:23, 62.05it/s]

 98%|█████████▊| 92910/94335 [12:15<00:22, 62.74it/s]

 98%|█████████▊| 92918/94335 [12:15<00:21, 65.35it/s]

 99%|█████

In [18]:
path

'/media/allen/mass/PAR/market1501/bounding_box_test/1488_c1s6_023021_02.jpg.jpg'

In [14]:
filename = path.split(branch+"/")[-1].replace("/","_")

In [16]:
filename.split(".")

['bounding_box_test_1485_c3s3_096319_04', 'jpg']

In [47]:
path.split("PAR/")


['/media/allen/mass/',
 'parse27k/cropped/sequences_log_15_8_2012_____16_28_45_image_00000180_0_rect.png']

### load data by build_loader

In [3]:
a, b = build_par_loader(cfg)

2019-08-25 00:23:10,795 logger INFO: => PAR loaded
2019-08-25 00:23:10,796 logger INFO: Dataset statistics:
2019-08-25 00:23:10,797 logger INFO:   ------------------------------
2019-08-25 00:23:10,797 logger INFO:   subset   | # class | # images
2019-08-25 00:23:10,798 logger INFO:   ------------------------------
2019-08-25 00:23:10,798 logger INFO:   train    |      13 |   185887
2019-08-25 00:23:10,799 logger INFO:   val      |      13 |    94335
2019-08-25 00:23:10,800 logger INFO:   ------------------------------


In [4]:
img, attrs = next(iter(a))

In [7]:
attrs[0]

tensor([ 1.,  0.,  0., -1.,  0.,  0.,  0., -1., -1.,  1.,  0., -1., -1.])

In [8]:
img, attrs = train_dataset[1]

In [6]:
img

In [7]:
attrs

[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1]

In [6]:
sampler.index_dic

defaultdict(list,
            {483: [0,
              154,
              514,
              1924,
              3016,
              5255,
              5863,
              6173,
              7541,
              8035,
              10008,
              10197,
              10440,
              10707,
              11006,
              11683,
              11698],
             585: [1,
              2,
              1572,
              2043,
              3381,
              3561,
              4756,
              5591,
              5612,
              5769,
              6423,
              8898,
              9964,
              10262,
              10726,
              11361,
              12229],
             721: [3,
              1030,
              1177,
              1340,
              4368,
              11048,
              12252,
              12401,
              12635,
              12859],
             303: [4, 256, 2400, 4592, 5266, 6406, 8913, 10211],
             661:

In [2]:
import torch
from model.OSNetv2 import osnet_x1_0
model = osnet_x1_0(751)
state = torch.load("/home/allen/Downloads/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth")

In [3]:
for key in state.keys():
    print("{:60}, {}".format(key, state[key].shape))

conv1.conv.weight                                           , torch.Size([64, 3, 7, 7])
conv1.bn.weight                                             , torch.Size([64])
conv1.bn.bias                                               , torch.Size([64])
conv1.bn.running_mean                                       , torch.Size([64])
conv1.bn.running_var                                        , torch.Size([64])
conv1.bn.num_batches_tracked                                , torch.Size([])
conv2.0.conv1.conv.weight                                   , torch.Size([64, 64, 1, 1])
conv2.0.conv1.bn.weight                                     , torch.Size([64])
conv2.0.conv1.bn.bias                                       , torch.Size([64])
conv2.0.conv1.bn.running_mean                               , torch.Size([64])
conv2.0.conv1.bn.running_var                                , torch.Size([64])
conv2.0.conv1.bn.num_batches_tracked                        , torch.Size([])
conv2.0.conv2a.conv1.weight          

In [4]:
model.load_state_dict(state)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
dummy = torch.ones(1,3,256,128)
model.eval()
output = model(dummy)
output.shape

torch.Size([1, 512])

In [11]:
state['model'].keys()

odict_keys(['backbone.data_norm.weight', 'backbone.data_norm.bias', 'backbone.data_norm.running_mean', 'backbone.data_norm.running_var', 'backbone.data_norm.num_batches_tracked', 'backbone.conv1.conv.weight', 'backbone.conv1.bn.weight', 'backbone.conv1.bn.bias', 'backbone.conv1.bn.running_mean', 'backbone.conv1.bn.running_var', 'backbone.conv1.bn.num_batches_tracked', 'backbone.bottleneck_1.0.conv1x1_in.conv.weight', 'backbone.bottleneck_1.0.conv1x1_in.bn.weight', 'backbone.bottleneck_1.0.conv1x1_in.bn.bias', 'backbone.bottleneck_1.0.conv1x1_in.bn.running_mean', 'backbone.bottleneck_1.0.conv1x1_in.bn.running_var', 'backbone.bottleneck_1.0.conv1x1_in.bn.num_batches_tracked', 'backbone.bottleneck_1.0.dwconv3x3.conv.weight', 'backbone.bottleneck_1.0.dwconv3x3.bn.weight', 'backbone.bottleneck_1.0.dwconv3x3.bn.bias', 'backbone.bottleneck_1.0.dwconv3x3.bn.running_mean', 'backbone.bottleneck_1.0.dwconv3x3.bn.running_var', 'backbone.bottleneck_1.0.dwconv3x3.bn.num_batches_tracked', 'backbone.b

In [1]:
from model.managers.trick_manager import TrickManager
from config.config_manager import _C as cfg
cfg.merge_from_file("./reid.yml")

model_manager = TrickManager(cfg)

I0711 13:39:08.929727 3353 model_manager.py:24] Evaluating model from /media/allen/mass/deep-learning-works/result_RMNet_cels_triplet_center_Adam_2019-07-11_10-45-09/weights/model_140_0.8477.pth
I0711 13:39:09.025811 3353 model_manager.py:83] opt_0 is skipped
I0711 13:39:09.026329 3353 model_manager.py:83] opt_1 is skipped
I0711 13:39:09.029359 3353 model_manager.py:64] odict_keys(['data_norm.weight', 'data_norm.bias', 'data_norm.running_mean', 'data_norm.running_var', 'data_norm.num_batches_tracked', 'conv1.conv.weight', 'conv1.bn.weight', 'conv1.bn.bias', 'conv1.bn.running_mean', 'conv1.bn.running_var', 'conv1.bn.num_batches_tracked', 'bottleneck_1.0.conv1x1_in.conv.weight', 'bottleneck_1.0.conv1x1_in.bn.weight', 'bottleneck_1.0.conv1x1_in.bn.bias', 'bottleneck_1.0.conv1x1_in.bn.running_mean', 'bottleneck_1.0.conv1x1_in.bn.running_var', 'bottleneck_1.0.conv1x1_in.bn.num_batches_tracked', 'bottleneck_1.0.dwconv3x3.conv.weight', 'bottleneck_1.0.dwconv3x3.bn.weight', 'bottleneck_1.0.dwc

I0711 13:39:09.030420 3353 model_manager.py:71] backbone.data_norm.weight                                    ...... skipped
I0711 13:39:09.031148 3353 model_manager.py:71] backbone.data_norm.bias                                      ...... skipped
I0711 13:39:09.031706 3353 model_manager.py:71] backbone.data_norm.running_mean                              ...... skipped
I0711 13:39:09.032211 3353 model_manager.py:71] backbone.data_norm.running_var                               ...... skipped
I0711 13:39:09.033802 3353 model_manager.py:71] backbone.data_norm.num_batches_tracked                       ...... skipped
I0711 13:39:09.034521 3353 model_manager.py:71] backbone.conv1.conv.weight                                   ...... skipped
I0711 13:39:09.035177 3353 model_manager.py:71] backbone.conv1.bn.weight                                     ...... skipped
I0711 13:39:09.036566 3353 model_manager.py:71] backbone.conv1.bn.bias                                       ...... skipped
I0711 13

I0711 13:39:09.066816 3353 model_manager.py:71] backbone.bottleneck_1.3.conv1x1_in.bn.bias                   ...... skipped
I0711 13:39:09.067223 3353 model_manager.py:71] backbone.bottleneck_1.3.conv1x1_in.bn.running_mean           ...... skipped
I0711 13:39:09.067686 3353 model_manager.py:71] backbone.bottleneck_1.3.conv1x1_in.bn.running_var            ...... skipped
I0711 13:39:09.068112 3353 model_manager.py:71] backbone.bottleneck_1.3.conv1x1_in.bn.num_batches_tracked    ...... skipped
I0711 13:39:09.068577 3353 model_manager.py:71] backbone.bottleneck_1.3.dwconv3x3.conv.weight                ...... skipped
I0711 13:39:09.069028 3353 model_manager.py:71] backbone.bottleneck_1.3.dwconv3x3.bn.weight                  ...... skipped
I0711 13:39:09.069532 3353 model_manager.py:71] backbone.bottleneck_1.3.dwconv3x3.bn.bias                    ...... skipped
I0711 13:39:09.069981 3353 model_manager.py:71] backbone.bottleneck_1.3.dwconv3x3.bn.running_mean            ...... skipped
I0711 13

I0711 13:39:09.106398 3353 model_manager.py:71] backbone.bottleneck_2.2.dwconv3x3.bn.running_mean            ...... skipped
I0711 13:39:09.106778 3353 model_manager.py:71] backbone.bottleneck_2.2.dwconv3x3.bn.running_var             ...... skipped
I0711 13:39:09.107162 3353 model_manager.py:71] backbone.bottleneck_2.2.dwconv3x3.bn.num_batches_tracked     ...... skipped
I0711 13:39:09.107559 3353 model_manager.py:71] backbone.bottleneck_2.2.conv1x1_out.conv.weight              ...... skipped
I0711 13:39:09.107945 3353 model_manager.py:71] backbone.bottleneck_2.2.conv1x1_out.bn.weight                ...... skipped
I0711 13:39:09.108327 3353 model_manager.py:71] backbone.bottleneck_2.2.conv1x1_out.bn.bias                  ...... skipped
I0711 13:39:09.108713 3353 model_manager.py:71] backbone.bottleneck_2.2.conv1x1_out.bn.running_mean          ...... skipped
I0711 13:39:09.109094 3353 model_manager.py:71] backbone.bottleneck_2.2.conv1x1_out.bn.running_var           ...... skipped
I0711 13

I0711 13:39:09.131678 3353 model_manager.py:71] backbone.bottleneck_2.6.conv1x1_in.bn.running_var            ...... skipped
I0711 13:39:09.132030 3353 model_manager.py:71] backbone.bottleneck_2.6.conv1x1_in.bn.num_batches_tracked    ...... skipped
I0711 13:39:09.132398 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.conv.weight                ...... skipped
I0711 13:39:09.132753 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.bn.weight                  ...... skipped
I0711 13:39:09.133098 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.bn.bias                    ...... skipped
I0711 13:39:09.133448 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.bn.running_mean            ...... skipped
I0711 13:39:09.133809 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.bn.running_var             ...... skipped
I0711 13:39:09.134159 3353 model_manager.py:71] backbone.bottleneck_2.6.dwconv3x3.bn.num_batches_tracked     ...... skipped
I0711 13

...

I0711 13:39:09.217402 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.conv.weight                ...... skipped
I0711 13:39:09.217849 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.bn.weight                  ...... skipped
I0711 13:39:09.218296 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.bn.bias                    ...... skipped
I0711 13:39:09.218746 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.bn.running_mean            ...... skipped
I0711 13:39:09.219191 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.bn.running_var             ...... skipped
I0711 13:39:09.219657 3353 model_manager.py:71] backbone.bottleneck_3.5.dwconv3x3.bn.num_batches_tracked     ...... skipped
I0711 13:39:09.220112 3353 model_manager.py:71] backbone.bottleneck_3.5.conv1x1_out.conv.weight              ...... skipped
I0711 13:39:09.220571 3353 model_manager.py:71] backbone.bottleneck_3.5.conv1x1_out.bn.weight                ...... skipped
I0711 13

I0711 13:39:09.247738 3353 model_manager.py:71] backbone.bottleneck_3.9.conv1x1_in.bn.weight                 ...... skipped
I0711 13:39:09.248176 3353 model_manager.py:71] backbone.bottleneck_3.9.conv1x1_in.bn.bias                   ...... skipped
I0711 13:39:09.248617 3353 model_manager.py:71] backbone.bottleneck_3.9.conv1x1_in.bn.running_mean           ...... skipped
I0711 13:39:09.249054 3353 model_manager.py:71] backbone.bottleneck_3.9.conv1x1_in.bn.running_var            ...... skipped
I0711 13:39:09.249487 3353 model_manager.py:71] backbone.bottleneck_3.9.conv1x1_in.bn.num_batches_tracked    ...... skipped
I0711 13:39:09.249930 3353 model_manager.py:71] backbone.bottleneck_3.9.dwconv3x3.conv.weight                ...... skipped
I0711 13:39:09.250346 3353 model_manager.py:71] backbone.bottleneck_3.9.dwconv3x3.bn.weight                  ...... skipped
I0711 13:39:09.250781 3353 model_manager.py:71] backbone.bottleneck_3.9.dwconv3x3.bn.bias                    ...... skipped
I0711 13

I0711 13:39:09.282192 3353 model_manager.py:71] backbone.bottleneck_4.2.dwconv3x3.bn.bias                    ...... skipped
I0711 13:39:09.282742 3353 model_manager.py:71] backbone.bottleneck_4.2.dwconv3x3.bn.running_mean            ...... skipped
I0711 13:39:09.283280 3353 model_manager.py:71] backbone.bottleneck_4.2.dwconv3x3.bn.running_var             ...... skipped
I0711 13:39:09.283850 3353 model_manager.py:71] backbone.bottleneck_4.2.dwconv3x3.bn.num_batches_tracked     ...... skipped
I0711 13:39:09.284394 3353 model_manager.py:71] backbone.bottleneck_4.2.conv1x1_out.conv.weight              ...... skipped
I0711 13:39:09.284927 3353 model_manager.py:71] backbone.bottleneck_4.2.conv1x1_out.bn.weight                ...... skipped
I0711 13:39:09.285449 3353 model_manager.py:71] backbone.bottleneck_4.2.conv1x1_out.bn.bias                  ...... skipped
I0711 13:39:09.286004 3353 model_manager.py:71] backbone.bottleneck_4.2.conv1x1_out.bn.running_mean          ...... skipped
I0711 13

I0711 13:39:09.316798 3353 model_manager.py:71] backbone.bottleneck_4.6.conv1x1_in.bn.running_mean           ...... skipped
I0711 13:39:09.317203 3353 model_manager.py:71] backbone.bottleneck_4.6.conv1x1_in.bn.running_var            ...... skipped
I0711 13:39:09.317599 3353 model_manager.py:71] backbone.bottleneck_4.6.conv1x1_in.bn.num_batches_tracked    ...... skipped
I0711 13:39:09.317993 3353 model_manager.py:71] backbone.bottleneck_4.6.dwconv3x3.conv.weight                ...... skipped
I0711 13:39:09.318406 3353 model_manager.py:71] backbone.bottleneck_4.6.dwconv3x3.bn.weight                  ...... skipped
I0711 13:39:09.318808 3353 model_manager.py:71] backbone.bottleneck_4.6.dwconv3x3.bn.bias                    ...... skipped
I0711 13:39:09.319224 3353 model_manager.py:71] backbone.bottleneck_4.6.dwconv3x3.bn.running_mean            ...... skipped
I0711 13:39:09.319699 3353 model_manager.py:71] backbone.bottleneck_4.6.dwconv3x3.bn.running_var             ...... skipped
I0711 13

I0711 13:39:09.374057 3353 model_manager.py:71] backbone.bottleneck_4.9.conv1x1_out.bn.running_var           ...... skipped
I0711 13:39:09.374450 3353 model_manager.py:71] backbone.bottleneck_4.9.conv1x1_out.bn.num_batches_tracked   ...... skipped
I0711 13:39:09.374831 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.conv.weight              ...... skipped
I0711 13:39:09.375193 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.bn.weight                ...... skipped
I0711 13:39:09.375587 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.bn.bias                  ...... skipped
I0711 13:39:09.375967 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.bn.running_mean          ...... skipped
I0711 13:39:09.376324 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.bn.running_var           ...... skipped
I0711 13:39:09.376698 3353 model_manager.py:71] backbone.bottleneck_4.10.conv1x1_in.bn.num_batches_tracked   ...... skipped
I0711 13

In [15]:
model_state = model_manager.model.state_dict()

In [16]:
'backbone.data_norm.weight' in model_state

True

In [17]:
torch.isnan(state['model']['backbone.data_norm.weight'])

tensor([0, 0, 0], device='cuda:0', dtype=torch.uint8)